<h2 style="border: 2px solid black;background-color: #ffffff;
background-image: linear-gradient(315deg, #ffffff 0%, #d7e1ec 74%);"><div style="text-align: center;height: 40px;line-height: 40px;">Publipostage 🚀:</div></h2>

In [15]:
# ----------------------------------------------------Librairies----------------------------------------------------------
import os
import pandas as pd
try:
    import win32com.client as win32
except Exception as e:
    os.system("pip install pywin32")
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, Markdown, clear_output, HTML
from datetime import datetime, timedelta    
from os import listdir
from os.path import isfile, join
import sys
try:
    import pyautogui
except Exception as e:
    os.system("pip install pyautogui")

In [16]:
# ----------------------------------------------------Fonctions----------------------------------------------------------
def send_mail(text, subject, recipient, mail_cc, attach_path):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient
    mail.Cc = mail_cc
    mail.Subject = subject
    mail.Attachments.Add(attach_path)
    mail.HtmlBody = text
    mail.Display(True)

def main(date_closing, date_reglement, subject_mail, cc_mail, text_mail, client_path, pdf_path):
    # Fichier souscripteurs
    pdf_files = [f for f in listdir(pdf_path) if isfile(join(pdf_path, f))]
    df_client = pd.read_excel(client_path, header=1).dropna(subset=['Numéro de compte'])
    status_df = pd.DataFrame([])
    for i, client in df_client.iterrows():
        copie_mail = cc_mail
        num_compte = client['Numéro de compte']
        mail_cgp = client['mail distributeur']
        mail_client = client['Email']
        pdf = ''.join([file for file in pdf_files if file == num_compte + '.pdf'])
        if pdf != '':
            attach_path = pdf_path + "\\" + pdf
            mail_cc = copie_mail + '; ' + mail_cgp
            send_mail(text_mail.format(str(date_closing), str(date_reglement)), subject_mail.format(str(date_closing)), mail_client, mail_cc, attach_path)
            status_df = pd.concat([status_df, pd.DataFrame([[num_compte, mail_client, 'OK']], columns=['Numeros', 'Mail Client', 'Status'])])
        else:
            status_df = pd.concat([status_df, pd.DataFrame([[num_compte, mail_client, 'NO']], columns=['Numeros', 'Mail Client', 'Status'])])
    return status_df

In [17]:
# ----------------------------------------------------Widgets----------------------------------------------------------
layout = widgets.Layout(flex='0 1 auto', height='30px', min_height='40px', width='auto')
# Date du closing
date_closing=widgets.DatePicker(description='Closing:',value=pd.to_datetime(datetime.today().date()))
# Date fin de reglement
date_reglement=widgets.DatePicker(description='Réglement:',value=pd.to_datetime(datetime.today().date()))
# Chemin
# client_path = r'\\192.168.68.9\Commun_IS\MDC\Service Client\3. Private Stars Selection II\2_Bulletin de souscription\tableau suivi souscripteurs.xlsx'
client_path = widgets.Text(value=r'C:\Users\joan.serfaty\Desktop\tableau suivi souscripteurs.xlsx', description='Fichier Excel:', layout=layout)
pdf_path = widgets.Text(value=r'\\192.168.68.9\Commun_IS\Gestion\Gestion diversifiée\PSS2\PASSIF\20220331_PSS2_CLOSING2', description='PDF:', layout=layout)

# cc
cc_mail = widgets.Text(value="""percilia.antunes@sanso-is.com; pierre.burnand@sanso-is.com; Estelle.COTTENCON@sanso-is.com""", description='Email en CC:', layout=layout)
# Sujet
subject_mail = widgets.Text(value="""Notice d’appel du 2ème closing - {0}""", description='Sujet : ', layout=layout)
# Body
text_mail = widgets.Text(value= """
Bonjour,<br><br>

Nous vous prions de trouver ci-joint la notice d’appel à la suite du 2ème closing du {0}.<br>
Merci de procéder au règlement au plus tard le {1}.
<p><i><u>Attention :</u> Le règlement doit être versé uniquement sur le RIB du FCPI Private Stars Selection 2 transmis dans cette notice. Ce RIB restera inchangé tout au long de la vie du fonds.</p></i>
Les équipes de Sanso IS restent à votre disposition pour toute information complémentaire.<br>
Bien cordialement, <br><br>
Sanso
""", description='Corps :', layout = layout)

# Button & Output
button = widgets.Button(description='Launch',button_style='success')
#button_stop = widgets.Button(description='Stop',button_style='danger')
out = widgets.Output()

def on_button_clicked(_):
      with out:
        dte_closing = pd.to_datetime(date_closing.value).date()
        dte_reglement = pd.to_datetime(date_reglement.value).date()
        copie_mail = cc_mail.value
        sub_mail = subject_mail.value
        body_mail = text_mail.value
        excel_path = client_path.value
        pdff_path = pdf_path.value
        tab = main(dte_closing, dte_reglement, sub_mail, copie_mail, body_mail, excel_path, pdff_path)
        clear_output(wait=True)
        tab

        
# We link the form and the output
button.on_click(on_button_clicked)

widgets.VBox([date_closing, date_reglement, subject_mail, cc_mail, text_mail, client_path, pdf_path, button, out])

In [18]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show/ Hide code."></form>''')


In [9]:
from IPython.core.display import display, HTML
HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_above()"><input type="submit" id="toggleButton" value="Run all Jupyter Cells"></form>''')